In [ ]:
import sqlite3
import matplotlib.pyplot as plt

# ბაზასთან დაკავშირება
conn = sqlite3.connect("students.sqlite")
cursor = conn.cursor()

# ცხრილების გამოჩნენა
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("ცხრილები ბაზაში:", cursor.fetchall())


In [ ]:
# ვკითხულობთ ყველა სტუდენტს, რომელთაც აქვთ მათემატიკაში 90-ზე მეტი ქულა
cursor.execute("SELECT gender, `math score` FROM Students WHERE `math score` > 90")
rows = cursor.fetchall()
for row in rows:
    print(row)

# ამ კოდი ბეჭდავს იმ სტუდენტებს, რომელთაც მათემატიკაში 90-ზე მეტი ქულა აქვთ


In [ ]:
# ჩანაწერის დამატება მომხმარებლის მიერ შეყვანილი მონაცემებით
gender = input("შეიყვანე სქესი: ")
race = input("შეიყვანე ეთნიკური ჯგუფი: ")
education = input("შეიყვანე მშობლის განათლების დონე: ")
lunch = input("შეიყვანე სადილი (standard/free/reduced): ")
prep = input("შეიყვანე ტესტის მომზადების კურსი (none/completed): ")
math = int(input("შეიყვანე მათემატიკის ქულა: "))
reading = int(input("კითხვის ქულა: "))
writing = int(input("წერის ქულა: "))

cursor.execute("""
INSERT INTO Students (
    gender, `race/ethnicity`, `parental level of education`, lunch,
    `test preparation course`, `math score`, `reading score`, `writing score`
) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""", (gender, race, education, lunch, prep, math, reading, writing))
conn.commit()

print("ჩანაწერი დამატებულია.")

# ეს კოდი ამატებს ახალ სტუდენტს ბაზაში


In [ ]:
# ჩანაწერის განახლება — math score კონკრეტული პირობით
target_gender = input("შეიყვანე სქესი: ")
new_math = int(input("შეიყვანე ახალი ქულა მათემატიკაში: "))

cursor.execute("UPDATE Students SET `math score` = ? WHERE gender = ?", (new_math, target_gender))
conn.commit()

print("განახლება შესრულებულია.")

# კოდი განაახლებს ყველა სტუდენტის მათემატიკის ქულას მითითებული სქესის მიხედვით


In [ ]:
# ჩანაწერის წაშლ
target_race = input("შეიყვანე ჯგუფი რომლის სტუდენტებიც წაიშლება ( напр. group A): ")
cursor.execute("DELETE FROM Students WHERE `race/ethnicity` = ?", (target_race,))
conn.commit()

print("ჩანაწერები წაიშალა მითითებული ჯგუფის სტუდენტებისთვის.")

# წაშლის ყველა სტუდენტს მითითებული race/ethnicity ჯგუფიდან


In [ ]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM Students", conn)

# 📊 დიაგრამა 1 — ჰისტოგრამა მათემატიკის ქულების
plt.hist(df['math score'], bins=10, color='skyblue')
plt.title("მათემატიკის ქულების განაწილება")
plt.xlabel("ქულა")
plt.ylabel("სტუდენტების რაოდენობა")
plt.grid(True)
plt.show()


In [ ]:
# 📊 დიაგრამა 2 — ბარის დიაგრამა სქესისა და საშუალო წერის ქულის მიხედვით
avg_scores = df.groupby('gender')['writing score'].mean()
avg_scores.plot(kind='bar', color='orange')
plt.title("საშუალო წერის ქულა სქესის მიხედვით")
plt.ylabel("ქულა")
plt.grid(axis='y')
plt.show()


In [ ]:
# 📊 დიაგრამა 3 — წრიული დიაგრამა ტესტის მომზადების კურსის კატეგორიებით
prep_counts = df['test preparation course'].value_counts()
prep_counts.plot(kind='pie', autopct='%1.1f%%', startangle=140)
plt.title("ტესტის მომზადების კურსის სტატისტიკა")
plt.ylabel("")
plt.show()

conn.close()
